In [1]:
# import required packages
import warnings
warnings.simplefilter("ignore")
import sys
sys.path.append("../src")
import os
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from cell_classification.application import Nimbus
from alpineer import io_utils
from cell_classification.viewer_widget import NimbusViewer
import tensorflow as tf

## 0: Set root directory and download example dataset
Here we are using the example data located in `/data/example_dataset/input_data`. To modify this notebook to run using your own data, simply change `base_dir` to point to your own sub-directory within the data folder. Set `base_dir`, the path to all of your imaging data (i.e. multiplexed images and segmentation masks). Subdirectory `nimbus_output` will contain all of the data generated by this notebook. In the following, we expect this folder structure:
```
|-- base_dir
|   |-- image_data
|   |   |-- fov_1
|   |   |-- fov_2
|   |-- segmentation
|   |   |-- deepcell_output
|   |-- cell_classification
```

In [2]:
# set up the base directory
base_dir = "E:/angelo_lab/data/TONIC/raw"

## 1: set file paths and parameters

### All data, images, files, etc. must be placed in the 'data' directory, and referenced via '../data/path_to_your_data'


In [3]:
# set up file paths
tiff_dir = os.path.join(base_dir, "image_data", "samples")
deepcell_output_dir = os.path.join(base_dir, "segmentation_data", "deepcell_output")
nimbus_output_dir = os.path.join(base_dir, "segmentation_data", "nimbus_output")

# Create nimbus output directory
os.makedirs(nimbus_output_dir, exist_ok=True)

# Check if paths exist
io_utils.validate_paths([base_dir, tiff_dir, deepcell_output_dir, nimbus_output_dir])

## 2: Set up input paths and the naming convention for the segmentation data
Store names of channels to exclude in the list below. Either predict all FOVs or specify manually the ones you want to apply nimbus on. The `segmentation_naming_convention` maps a FOV path to the according instance segmentation output path. Please make sure, that `segmentation_naming_convention` returns the path to the correct cell segmentation output. 

In [4]:
# define the channels to exclude
exclude_channels = [
    'H3K9ac', 'H3K27me3', "Au", "Fe", "Noodle", "Ca", "CD11c_nuc_exclude", "CK17_smoothed",
    "Collagen1", "ECAD_smoothed", "FOXP3_nuc_include", "SMA", "VIM"
]

# either get all fovs in the folder...
fov_names = os.listdir(tiff_dir)
# ... or optionally, select a specific set of fovs manually
# fovs = ["fov0", "fov1"]

fov_paths = [os.path.join(tiff_dir, fov_name) for fov_name in fov_names]

# define the naming convention for the segmentation data
def segmentation_naming_convention(fov_path):
    """Prepares the path to the segmentation data for a given fov
    Args:
        fov_path (str): path to fov
    Returns:
        seg_path (str): paths to segmentation fovs
    """
    fov_name = os.path.basename(fov_path)
    return os.path.join(
        deepcell_output_dir, fov_name + "_feature_0.tif"
    )

# test segmentation naming convention
if os.path.exists(segmentation_naming_convention(fov_paths[0])):
    print("Segmentation data exists for fov 0 and naming convention is correct")
else:
    print("Segmentation data does not exist for fov 0 or naming convention is incorrect")

Segmentation data exists for fov 0 and naming convention is correct


## 3: Load model and initialize Nimbus application
The following code initializes the Nimbus application and loads the model checkpoint. The model was trained on a diverse set of tissues, protein markers, imaging platforms and cell types and doesn't need re-training.

In [5]:
nimbus = Nimbus(
    fov_paths=fov_paths,
    segmentation_naming_convention=segmentation_naming_convention,
    output_dir=nimbus_output_dir,
    exclude_channels=exclude_channels,
    save_predictions=True,
)

# check if all inputs are valid
nimbus.check_inputs()

Loaded weights from E:\angelo_lab\cell_classification\checkpoints\halfres_512_checkpoint_160000.h5
All inputs are valid.


## 4: Prepare normalization dictionary 
The next step is to iterate through all the fovs and calculate the 0.999 marker expression quantile for each marker individually. This is used for normalizing the marker expressions prior to predicting marker confidence scores with our model. You can set `n_subset` to estimate the quantiles on a small subset of the data and you can set `multiprocessing` to speed up computation.

In [6]:
nimbus.prepare_normalization_dict(
    n_subset=5,
    multiprocessing=True,
    overwrite=True
)

Iterate over fovs...


  0%|          | 0/5 [00:00<?, ?it/s]

## 5: Make predictions with the model
Nimbus will iterate through your samples and store predictions and a file named `nimbus_cell_table.csv` that contains the mean-per-cell predicted marker confidence scores in the sub-directory called `nimbus_output`.

In [7]:
cell_table = nimbus.predict_fovs()

Available GPUs:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
Predictions will be saved in E:/angelo_lab/data/TONIC/raw\segmentation_data\nimbus_output
Iterating through fovs will take a while...


  0%|          | 0/5 [00:00<?, ?it/s]

## 6: View multiplexed channels and Nimbus predictions side-by-side
Select an FOV and one marker image per channel to inspect the imaging data and associated Nimbus predictions

In [12]:
import os
import ipywidgets as widgets
from IPython.display import display
from io import BytesIO
from skimage import io
from copy import copy

class NimbusViewer(object):
    def __init__(self, input_dir, output_dir, img_width='600px'):
        """Viewer for Nimbus application.
        Args:
            input_dir (str): Path to directory containing individual channels of multiplexed images.
            output_dir (str): Path to directory containing output of Nimbus application.
            img_width (str): Width of images in viewer.
        """
        self.image_width = img_width
        self.input_dir = input_dir
        self.output_dir = output_dir
        self.fov_names = [os.path.basename(p) for p in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir, p))]
        self.update_button = widgets.Button(description="Update Image")
        self.update_button.on_click(self.update_button_click)

        self.fov_select = widgets.Select(
            options=self.fov_names,
            description='FOV:',
            disabled=False
        )
        self.fov_select.observe(self.select_fov, names='value')

        self.red_select = widgets.Select(
            options=[],
            description='Red:',
            disabled=False
        )
        self.green_select = widgets.Select(
            options=[],
            description='Green:',
            disabled=False
        )
        self.blue_select = widgets.Select(
            options=[],
            description='Blue:',
            disabled=False
        )
        self.input_image = widgets.Image()
        self.output_image = widgets.Image()

    def select_fov(self, change):
        fov_path = os.path.join(self.output_dir, self.fov_select.value)
        channels = [ch for ch in os.listdir(fov_path) if os.path.isfile(os.path.join(fov_path, ch))]
        self.red_select.options = channels
        self.green_select.options = channels
        self.blue_select.options = channels

    def create_composite_image(self, path_dict):
        output_image = []
        img = None
        for k, p in path_dict.items():
            if p:
                img = io.imread(p)
                output_image.append(img)
            if p is None:
                non_none = [p for p in path_dict.values() if p]
                if not img:
                    img = io.imread(non_none[0])
                output_image.append(img*0)

        composite_image = np.stack(output_image, axis=-1)
        return composite_image

    def layout(self):
        channel_selectors = widgets.VBox([
            self.red_select,
            self.green_select,
            self.blue_select
        ])
        self.input_image.layout.width = self.image_width
        self.output_image.layout.width = self.image_width
        viewer_html = widgets.HTML("<h2>Select files</h2>")
        input_html = widgets.HTML("<h2>Input</h2>")
        output_html = widgets.HTML("<h2>Nimbus Output</h2>")

        layout = widgets.HBox([
            widgets.VBox([
                viewer_html,
                self.fov_select,
                channel_selectors,
                self.update_button
            ]),
        widgets.VBox([
            input_html,
            self.input_image
        ]),
        widgets.VBox([
            output_html,
            self.output_image
        ])
        ])
        display(layout)

    def search_for_similar(self, select_value):
        in_f_path = os.path.join(self.input_dir, self.fov_select.value)
        # search for similar filename in in_f_path
        in_f_files = [f for f in os.listdir(in_f_path) if os.path.isfile(os.path.join(in_f_path, f))]
        similar_path = None
        for f in in_f_files:
            if select_value.split(".")[0]+"." in f:
                similar_path = os.path.join(self.input_dir, self.fov_select.value, f)
        return similar_path

    def update_img(self, image_viewer, composite_image):
        # Convert composite image to bytes and assign it to the output_image widget
        with BytesIO() as output_buffer:
            io.imsave(output_buffer, composite_image, format="png")  # Save as PNG (you can change the format)
            output_buffer.seek(0)
            image_viewer.value = output_buffer.read()

    def update_composite(self):
        path_dict = {
            "red": None,
            "green": None,
            "blue": None
        }
        in_path_dict = copy(path_dict)
        if self.red_select.value:
            path_dict["red"] = os.path.join(self.output_dir, self.fov_select.value, self.red_select.value)
            in_path_dict["red"] = self.search_for_similar(self.red_select.value)
        if self.green_select.value:
            path_dict["green"] = os.path.join(self.output_dir, self.fov_select.value, self.green_select.value)
            in_path_dict["green"] = self.search_for_similar(self.green_select.value)
        if self.blue_select.value:
            path_dict["blue"] = os.path.join(self.output_dir, self.fov_select.value, self.blue_select.value)
            in_path_dict["blue"] = self.search_for_similar(self.blue_select.value)
        non_none = [p for p in path_dict.values() if p]
        if not non_none:
            return
        composite_image = self.create_composite_image(path_dict)
        in_composite_image = self.create_composite_image(in_path_dict)
        in_composite_image = in_composite_image / np.quantile(in_composite_image, 0.999, axis=(0,1))
        in_composite_image = np.clip(in_composite_image*255, 0, 255).astype(np.uint8)
        # update image viewers
        self.update_img(self.input_image, in_composite_image)
        self.update_img(self.output_image, composite_image)

    def update_button_click(self, button):
        self.update_composite()
    
    def display(self):
        self.select_fov(None)
        self.layout()
        self.update_composite() 


In [13]:
viewer = NimbusViewer(input_dir=tiff_dir, output_dir=nimbus_output_dir)
viewer.display()

TypeError: select_fov() takes 1 positional argument but 2 were given